In [1]:
%%html
<style> 
table {margin-left: 0 !important;}
table td, table th, table tr {text-align:left !important;}
</style>

<div>
<img src="attachment:Bovag.jpg" width="400">
</div>

# Multiple Binomial Logistic Regression Models - Groepsopdracht voor werkcollege

## Casus
De Hogeschool Utrecht (HU) heeft zo’n 4.000 werknemers in dienst. Elk jaar vertrekt echter zo’n 15% van de werknemers en de corporate HRM afdeling heeft de taak om deze werknemers te vervangen. Het kost veel tijd en geld om geschikte werknemers te vinden, omdat er sprake is van een krappe arbeidsmarkt (er zijn meer vacatures dan werkzoekenden). 

Daarom wil de HU investeren in het *behouden* van werknemers, om zo het vertrekpercentage te verlagen. Het management heeft daarom aan het People Analytics team gevraagd te onderzoeken welke factoren het vertrek van werknemers kunnen verklaren, zodat gerichte interventies kunnen worden ingezet om werknemers te behouden. Jullie werken voor het People Analytics team en gaan onderzoeken welke factoren vertrek voorspellen. Dat doen jullie op basis van een dataset met daarin 4.410 (oud-)werknemers van de HU.

Jullie gaan een **hiërarchisch regressiemodel** bouwen en passen daarbij **het principe van parsimony** toe om te komen tot het meest simpele voorspellend model. Op basis van eerdere onderzoeken naar oorzaken van vertrek bouwen jullie het multiple regressie model in drie stappen op (of: 3 modellen). Hieronder staat toegelicht welke predictoren in elk model worden toegevoegd.  

Vertrek (Y) wordt gemeten als *de medewerker is in het afgelopen jaar vertrokken (ja/nee)*, heeft als variabelenaam ***Attrition_rec*** en is van *nominaal* meetniveau.

In onderstaande tabel zie je welke predictoren (X) je in welk model moet opnemen:


| Model | Predictor (X) | Variabelenaam | Meetniveau |
| :--- | :--- | :--- | :--- |
| 1 | Leeftijd | Age | Ratio |
| 1 | Opleidingsniveau | Education | Ordinaal |
| 1 | Geslacht | Gender | Nominaal | 
| 1 | Aantal organisaties waarvoor de werknemer gewerkt heeft | NumCompaniesWorked | Ratio | 
| 1 | Jaren werkervaring | TotalWorkingYears | Ratio |
| 2 | Maandsalaris | MonthlyIncome| Ratio |
| 2 | Reisafstand in kilometers | DistanceFromHome | Ratio |
| 2 | Jaren sinds laatste promotie | YearsSinceLastPromotion | Ratio |
| 2 | Performancescore | PerformanceRating | Interval |
| 2 | Aantal jaren werkzaam onder huidige manager | YearsWithCurrManager | Ratio |
| 3 | Werktevredenheid | JobSatisfaction | Interval |
| 3 | Werk-privé balans | WorkLifeBalance | Interval |
| 3 | Werkbetrokkenheid | JobInvolvement | Interval |

**Voetnoten bij tabel**
1. *Maandsalaris*: Het maandsalaris wordt weergegeven in roepies (Indiase munteendheid). Dat is natuurlijk raar, wij als HU-docenten krijgen gewoon in euro’s uitbetaald. We hadden een creatieve reden kunnen bedenken waarom hier roepies staan, maar jullie snappen de echte reden vast wel: het is een fictief bestand, gemaakt door Indiase HRM docent, waarvan wij met toestemming dankbaar gebruik van maken voor onze eigen fictieve casus. 
2. *Interval variabelen*: Deze schalen zijn eigenlijk van ordinaal meetniveau. In de praktijk worden schalen van ordinaal meetniveau met een structuur die lijkt op een Likert-schaal vaak behandeld als schalen van interval niveau. Dat doen wij hier ook. Daarop is uiteraard kritiek te leveren, maar die negeren wij voor deze opdracht voor ons gemak even.

We gaan nu R code runnen in een notebook, daarvoor moeten we gebruik maken van het package rpy2. Die importeren we, en door die te laden met %load_ext rpy2.ipython kunnen we R code runnen in de cellen van dit notebook. Wel moet je dan je cell laten beginnen met %%R. Doe je dat niet, dan kun je gewoon python-code runnen.


In [ ]:
import rpy2.rinterface

%load_ext rpy2.ipython

We installeren de benodigde R-packages en importeren ze. Importeren gaat in R in met de library() functie.

In [ ]:
%%R
install.packages('DescTools')
install.packages('caret')
library(dplyr)
library(DescTools)
library(caret)

We gaan nu de data inladen, die staat in General_data.csv. Deze moet je eerst uploaden naar colab. Klik daarvoor hiernaast op het 'folder'-icoon en kies voor bestand uploaden (pagina met een pijltje omhoog). Upload General_data.csv en run onderstaande cel.

In [ ]:
%%R
hrdata <- read.csv('./General_data.csv', sep =";")
head(hrdata)

Om de analyses te runnen moeten we de afhankelijke variable (Attrition) omzetten naar het type 'factor'. Je hoeft niet te weten hoe dat moet, maar dit wordt voor je gedaan in de onderstaande cell.
Ook worden de missing values verwijderd, met na.omit().

In [ ]:
%%R
#maak factor van afhankele variabele
hrdata <- hrdata %>% mutate(Attrition = ifelse(Attrition=="No", 0, 1))
hrdata$Attrition <- as.factor(hrdata$Attrition)
#drop nas
hrdata <- na.omit(hrdata)

## Stap 1: Dummy variabelen maken 

In stap 2 worden de variabelen Education en Gender toegevoegd. Dit zijn categorische variabelen waarvan eerst dummy variabelen worden gemaakt. 
- Maak de dummy variabelen aan met Education type ‘Bachelor’ als baseline categorie. Oftewel, maak vier dummy variabelen aan: Below_college, College, Master en Doctor. 
- Maak de dummy variabelen aan voor vrouwen, genaamd Female. 
  

Je hoeft niet zelf uit te zoeken hoe dat in R moet, je kunt onderstaande cellen runnen om de dummies aan te maken.

In [ ]:
%%R
#dummy for education
hrdata$Education <- as.factor(hrdata$Education)
hr_edu = hrdata[c("Education")]
dmy <- caret::dummyVars(" ~ .", data = hr_edu)
hr_edu = as.data.frame(predict(dmy, newdata = hr_edu))
hrdata <- cbind(hrdata, hr_edu[c("Education.2","Education.3","Education.4","Education.5")])
rm(hr_edu)

In [ ]:
%%R
#dummy for gender
hr_gender = hrdata[c("Gender")]
dmy <- caret::dummyVars(" ~ .", data = hr_gender)
hr_gender = as.data.frame(predict(dmy, newdata = hr_gender))
hrdata <- cbind(hrdata, hr_gender[c("GenderFemale")])
rm(hr_gender)

## Stap 2 - Multiple Binomial Logistic Regression Models

Voer de multiple binomial logistic regression models uit in R en beantwoord de vragen. 

In [ ]:
%%R
#train eerste model
null_model = glm(formula = Attrition ~ 1, data = hrdata, binomial("logit"))
model1 = glm(formula = Attrition ~ Age + Education.2 + Education.3 + Education.4 + Education.5 +
                GenderFemale+NumCompaniesWorked + TotalWorkingYears,
              data = hrdata, family = binomial("logit"))
odds_ratio = exp(model1$coefficients)
exp(confint(model1))

In [ ]:
%%R
#Compute pseudo R-squares
DescTools::PseudoR2(model1, which = c("CoxSnell","Nagelkerke"))

In [ ]:
%%R
summary(model1)


In [ ]:
%%R 
anova(null_model, model1, test = "Chisq")

In [ ]:
%%R
#model 2
model2 = glm(formula = Attrition ~ Age + Education.2 + Education.3 + Education.4 + Education.5 +
                GenderFemale+NumCompaniesWorked + TotalWorkingYears + MonthlyIncome + DistanceFromHome +
                YearsSinceLastPromotion + PerformanceRating + YearsWithCurrManager,
              data = hrdata, family = binomial("logit"))
summary(model2)


In [ ]:
%%R
print(DescTools::PseudoR2(model2, which = c("CoxSnell","Nagelkerke")))

In [ ]:
%%R
#Compute chi square
print(anova(model1, model2, test = "Chisq"))
print(anova(null_model, model2, test = "Chisq"))


In [ ]:
%%R
#model 3
formula = Attrition ~ Age + Education.2 + Education.3 + Education.4 + Education.5 +
  GenderFemale+NumCompaniesWorked + TotalWorkingYears + MonthlyIncome + DistanceFromHome +
  YearsSinceLastPromotion + PerformanceRating + YearsWithCurrManager +
  JobSatisfaction + WorkLifeBalance + JobInvolvement

model3 = glm(formula = formula,
              data = hrdata, family = binomial("logit"))

print(summary(model3))


In [ ]:
%%R
exp(cbind("Odds ratio" = coef(model3), confint.default(model3, level = 0.95)))

In [ ]:
%%R
DescTools::PseudoR2(model3, which = c("CoxSnell","Nagelkerke"))

In [ ]:
%%R
print(anova(model2, model3, test = "Chisq"))
print(anova(null_model, model3, test = "Chisq"))

### Vraag 1
*Interpreteer de pseudo R squares en concludeer welk model het beste het vertrek van werknemers voorspelt én hoe goed dit model het vertrek van werknemers voorspelt.* 

Typ hier je antwoord

### Vraag 2
*Interpreteer de Chi squares m.b.t. verbetering ten opzichte van vorige modellen en trek een passende conclusie over de model fit.* 

Typ hier je antwoord

### Vraag 3
*Interpreteer voor model 3 per predictor, indien relevant, de odds ratio en significantie (o.b.v. de confidence intervals) zeer nauwkeurig.*

Typ hier je antwoord

## Stap 3 - Model Parsimony

- Verwijder alle *niet significante predictoren* uit model 3 en run het model opnieuw. 
- Bereken vervolgens de AIC voor model 3 met álle predictoren *en* de AIC voor model 3 met álleen significante predictoren. 
- Beantwoord daarna onderstaande vragen.   

In [ ]:
%%R
#model 4 wordt berekend met enkel de significante predictoren
model4 = glm(formula = Attrition ~ Age + NumCompaniesWorked + TotalWorkingYears + MonthlyIncome +
               YearsSinceLastPromotion + YearsWithCurrManager +
               JobSatisfaction + WorkLifeBalance,
             data = hrdata, family = binomial("logit"))


print(AIC(model3))
print(AIC(model4))


### Vraag 1
*Leidt het verwijderen van niet significante predictoren daadwerkelijk tot een meer ‘parsimonious’ model? Licht je antwoord toe.* 

Typ hier je antwoord

## Stap 4 - Conclusies en advies

*Trek conclusies en breng op basis van bovenstaande resultaten een advies uit aan de opdrachtgevers van de HU.* 

Zorg ervoor dat je conclusies en advies aan de volgende criteria voldoen:
- Conclusies: Geef een korte beschrijving van resultaten in **‘lekentaal’** (te begrijpen voor iemand zonder veel kennis van statistiek) en verwerk daarin ook de verklaarde variantie. Betrek hierin ook de model parsimomy.
- Advies: Geef concrete aanbevelingen. Schrijf bijvoorbeeld niet alleen op ‘er is vervolgonderzoek nodig’, of ‘interventies moeten gericht zijn op variabele Y’, maar geef ook concrete suggesties. 

Typ hier jullie conclusies in lekentaal

Typ hier jullie advies